This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [2]:
from dotenv import load_dotenv
load_dotenv()
from langchain.llms import OpenAI
import os




In [27]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

# Step 1: Define the prompt template
real_estate_prompt = PromptTemplate(
    input_variables=["region", "num_listings", "output_format"],
    template="""
Generate a diverse and realistic set of {num_listings} real estate listings in {region}.
Each listing should contain the following fields:

- Property ID  
- Title  
- Address (Street, City, Province)  
- Price (CAD)  
- Bedrooms  
- Bathrooms  
- Area (sqft)  
- Type (e.g., Detached House, Condo, Apartment, Townhouse, Commercial Space, Cottage, Bungalow)  
- Status (For Sale, Sold, Pending, For Rent)  
- Description  
- Image URL  
- Agent Name  
- Agent Phone  
- Agent Email  

Ensure:
- Listings are realistic for the Canadian real estate market
- Data uses Canadian locations and formatting
- Output should be structured as {output_format}

Begin generating the listings now.
"""
)
# model_name = "gpt-3.5-turbo"
# Step 2: Initialize the LLM

llm = OpenAI(
    model_name="gpt-4o",
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Step 3: Create the LLM chain
real_estate_chain = LLMChain(
    llm=llm,
    prompt=real_estate_prompt
)

# Step 4: Run the chain with input values
response = real_estate_chain.run({
    "region": "Canada",
    "num_listings": "10",
    "output_format": "a CSV-formatted table"
})

# Output the generated listings
print(response)
# save the response to a text file
with open("real_estate_listings.txt", "w") as f:
    f.write(response)

with open("real_estate_listings.csv", "w") as f:
    f.write(response)





```csv
Property ID,Title,Address,Price (CAD),Bedrooms,Bathrooms,Area (sqft),Type,Status,Description,Image URL,Agent Name,Agent Phone,Agent Email
101,"Charming Family Home","123 Maple St, Toronto, ON",850000,4,3,2300,"Detached House","For Sale","A spacious family home located in a quiet neighborhood, featuring a large backyard and modern finishes.","https://example.com/image1.jpg","Emily Clark","416-555-1234","emily.clark@example.com"
102,"Luxury Downtown Condo","456 Bay St, Vancouver, BC",1150000,2,2,1100,"Condo","For Sale","A luxurious condo in the heart of downtown Vancouver with stunning city views and top-notch amenities.","https://example.com/image2.jpg","James Harris","604-555-5678","james.harris@example.com"
103,"Cozy Suburban Bungalow","789 Pine Ave, Calgary, AB",450000,3,1,1400,"Bungalow","Pending","This cozy bungalow is perfect for small families, located in a friendly suburban community.","https://example.com/image3.jpg","Sarah Thompson","403-555-9876","sarah.thompson@exampl